## Libs

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.metrics import 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from functools import partial
from itertools import chain

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

%matplotlib inline
np.random.seed(1)

## Build the Model

In [3]:
class q_model(nn.Module):
    def __init__(self, 
                 quantiles, 
                 in_shape=1,  
                 dropout=0.5):     
        super().__init__()
        self.quantiles = quantiles
        self.num_quantiles = len(quantiles)
        
        self.in_shape = in_shape
        self.out_shape = len(quantiles)
        self.dropout = dropout
        self.build_model()
        self.init_weights()
        
    def build_model(self): 
        # self.base_model = nn.Sequential(
        #     nn.Linear(self.in_shape, 64),
        #     nn.ReLU(),
        #     # nn.BatchNorm1d(64),
        #     nn.Dropout(self.dropout),
        #     nn.Linear(64, 64),
        #     nn.ReLU(),
        #     # nn.BatchNorm1d(64),
        #     nn.Dropout(self.dropout),
        # )
        self.base_model = nn.LSTM(3, 128, 2, batch_first=True)#input in 3 
        final_layers = [
            nn.Linear(128, 15) for _ in range(len(self.quantiles))#output in 15 
        ]
        self.final_layers = nn.ModuleList(final_layers)
        
    def init_weights(self):
        for m in chain(self.final_layers):
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight)
                nn.init.constant_(m.bias, 0)        
        
    # def forward(self, x):
    #     tmp_ = self.base_model(x)
    #     return torch.cat([layer(tmp_) for layer in self.final_layers], dim=1)
    
    def forward(self, x):
        out, _ = self.base_model(x)
        #out = self.final_layers(out[:, -1, :])  # Используйте только последний временной шаг для предсказания
        return torch.stack([layer(out[:, -1, :]) for layer in self.final_layers], dim=1)

In [4]:
class QuantileLoss(nn.Module):
    def __init__(self, quantiles):
        super().__init__()
        self.quantiles = quantiles
        
    def forward(self, preds, target):
        assert not target.requires_grad
        assert preds.size(0) == target.size(0)
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = target - preds[:, i]
            losses.append(torch.max((q-1) * errors, q * errors).unsqueeze(1))
        loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
        return loss

In [5]:
import tqdm
class Learner:
    def __init__(self, model, optimizer_class, loss_func, device='cpu'):
        self.model = model.to(device)
        self.optimizer = optimizer_class(self.model.parameters())
        self.loss_func = loss_func.to(device)
        self.device = device
        self.loss_history = []
        
    def fit(self, x, y, epochs, batch_size):
        self.model.train()
        for e in tqdm.tqdm(range(epochs)):
            shuffle_idx = np.arange(x.shape[0])
            np.random.shuffle(shuffle_idx)
            x = x[shuffle_idx]
            y = y[shuffle_idx]
            epoch_losses = []
            for idx in range(0, x.shape[0], batch_size):
                self.optimizer.zero_grad()
                batch_x = torch.from_numpy(
                    x[idx : min(idx + batch_size, x.shape[0]),:]
                ).float().to(self.device).requires_grad_(False)
                #print(batch_x)
                #print(type(batch_x))
                batch_y = torch.from_numpy(
                    y[idx : min(idx + batch_size, y.shape[0])]
                ).float().to(self.device).requires_grad_(False)
                preds = self.model(batch_x)
                loss = self.loss_func(preds, batch_y)
                loss.backward()
                self.optimizer.step()
                epoch_losses.append(loss.cpu().detach().numpy())                                
            epoch_loss =  np.mean(epoch_losses)
            self.loss_history.append(epoch_loss)
            print("Epoch {}: {}".format(e+1, epoch_loss))
                
    def predict(self, x, mc=False):
        if mc:
            self.model.train()
        else:
            self.model.eval()
        return self.model(torch.from_numpy(x).float().to(self.device).requires_grad_(False)).cpu().detach().numpy()

## Load data

In [9]:
y_smp_test = np.load('data/y_smp_test.npy')

In [12]:
y_smp_test.shape

(100000, 200, 3)

In [15]:
y_test = torch.Tensor(y_smp_test)
test_dataset = TensorDataset(y_test)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

## Predict

In [6]:
import pickle
filename = "learner.class"
filehandler = open(filename, 'rb') 
learner_load = pickle.load(filehandler)

In [28]:
with torch.no_grad():
    all_output_test = []
    for batch in test_loader:
        inputs_test= batch[0]
        tmp = learner_load.predict(inputs_test.numpy())
        all_output_test.append(tmp)
    final_output_test = np.concatenate(all_output_test)

c:\Users\Anty\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


In [31]:
final = np.transpose(final_output_test, (0, 2, 1))

In [37]:
final.shape

(100000, 15, 5)

In [39]:
final2 = final.copy()
new_column = final2[:, :, 2]
arr_with_new_column = np.insert(final2, 0, new_column, axis=2)

In [40]:
arr_with_new_column.shape

(100000, 15, 6)

In [30]:
final_output_test.shape
type(final_output_test)

numpy.ndarray

In [13]:
tmp = learner_load.predict(y_smp_test)

c:\Users\Anty\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\rnn.py:879: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.54 GiB. GPU 0 has a total capacty of 4.00 GiB of which 1.75 GiB is free. Of the allocated memory 661.52 MiB is allocated by PyTorch, and 4.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
tmp.shape